In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import sys
sys.path.insert(0, '/home/tomek/ib_tools/')
from indicators import get_signals
from utils import chart_price, plot, perf, v_backtester, c_backtester, perf_var

/home/tomek/.virtualenvs/ib/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
data = pd.read_pickle('freeze/live/all_bars_df_NQM0.pickle')

In [3]:
data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-05-12 00:00:00,9282.00,9288.75,9280.75,9283.75,130,9285.450000,90
2020-05-12 00:00:30,9283.00,9286.25,9281.50,9286.25,35,9283.475000,30
2020-05-12 00:01:00,9287.00,9289.00,9284.25,9284.25,91,9286.475000,73
2020-05-12 00:01:30,9285.25,9287.75,9284.75,9287.75,23,9285.925000,22
2020-05-12 00:02:00,9287.50,9289.25,9287.00,9288.25,34,9288.150000,30
...,...,...,...,...,...,...,...
2020-05-25 17:27:00,9528.00,9528.25,9527.75,9528.25,30,9528.033333,22
2020-05-25 17:27:30,9528.25,9528.25,9526.50,9527.00,53,9527.047170,33
2020-05-25 17:28:00,9526.50,9527.75,9526.50,9527.50,21,9527.130952,19


In [9]:
df=data.loc['2020-05-25 16:46:30':].head(50)
df

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-05-25 16:46:30,9524.00,9524.00,9523.00,9523.75,44,9523.471591,26
2020-05-25 16:47:00,9524.00,9524.00,9523.25,9523.50,20,9523.675000,14
2020-05-25 16:47:30,9523.50,9524.00,9523.50,9523.50,14,9523.642857,13
2020-05-25 16:48:00,9523.75,9524.50,9523.25,9523.25,36,9523.979167,22
2020-05-25 16:48:30,9523.00,9523.25,9522.75,9522.75,22,9523.022727,19
2020-05-25 16:49:00,9522.50,9523.00,9522.50,9522.75,14,9522.714286,11
2020-05-25 16:49:30,9522.50,9522.75,9522.25,9522.75,10,9522.500000,9
2020-05-25 16:50:00,9522.75,9523.50,9522.50,9523.25,33,9522.969697,16
2020-05-25 16:50:30,9523.50,9524.25,9523.50,9524.00,34,9523.867647,22


In [5]:
df.open[0], df.high.max(), df.low.min(), df.close[-1], df.volume.sum()

(9521.0, 9527.75, 9515.75, 9521.0, 2721)

In [6]:
data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-05-12 00:00:00,9282.00,9288.75,9280.75,9283.75,130,9285.450000,90
2020-05-12 00:00:30,9283.00,9286.25,9281.50,9286.25,35,9283.475000,30
2020-05-12 00:01:00,9287.00,9289.00,9284.25,9284.25,91,9286.475000,73
2020-05-12 00:01:30,9285.25,9287.75,9284.75,9287.75,23,9285.925000,22
2020-05-12 00:02:00,9287.50,9289.25,9287.00,9288.25,34,9288.150000,30
...,...,...,...,...,...,...,...
2020-05-25 17:27:00,9528.00,9528.25,9527.75,9528.25,30,9528.033333,22
2020-05-25 17:27:30,9528.25,9528.25,9526.50,9527.00,53,9527.047170,33
2020-05-25 17:28:00,9526.50,9527.75,9526.50,9527.50,21,9527.130952,19


In [7]:
data.loc['2020-05-21 10:44:30':].head(50)

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-05-21 10:44:30,9436.00,9436.00,9433.75,9434.50,48,9434.850,39
2020-05-21 10:45:00,9434.50,9435.25,9433.50,9433.75,40,9434.350,33
2020-05-21 10:45:30,9433.50,9434.00,9430.75,9430.75,105,9432.700,60
2020-05-21 10:46:00,9430.75,9430.75,9428.75,9430.50,93,9430.050,72
2020-05-21 10:46:30,9430.50,9432.00,9430.50,9431.75,35,9431.350,31
2020-05-21 10:47:00,9431.50,9431.50,9429.50,9429.50,49,9430.500,41
2020-05-21 10:47:30,9429.25,9429.50,9428.25,9429.50,31,9429.100,29
2020-05-21 10:48:00,9429.75,9431.75,9429.75,9431.00,26,9430.900,21
2020-05-21 10:48:30,9431.25,9432.25,9431.25,9432.25,15,9431.625,11


In [8]:
data[data.index.duplicated()]

,open,high,low,close,volume,average,barCount
date,,,,,,,
